In [1]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split,cross_validate, KFold, StratifiedKFold, GridSearchCV # 학습, 테스트set 구분
import sklearn.metrics as mt # 성능지표를 계산하기 위해 import
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

import warnings
warnings.filterwarnings('ignore')

In [2]:
df  = pd.read_csv('/content/drive/MyDrive/Dacon/train.csv')
test  = pd.read_csv('/content/drive/MyDrive/Dacon/test.csv')

In [3]:
train = df.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)
train = train.fillna(0) #test 결측치 0으로 채우기
test = test.fillna(0)

In [4]:
train.isna().sum()[train.isna().sum() != 0]

Series([], dtype: int64)

In [5]:
test.isna().sum()[test.isna().sum() != 0]

Series([], dtype: int64)

In [6]:
#train을 target과 feature로 나눠줍니다.
train_x=train.drop(['nerdiness'], axis=1)
train_y=train['nerdiness']

In [7]:
# 학습, 테스트 데이터 분리 (0.7:0.3)
seed = 2022
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=seed, stratify = train_y)

# 최적의 n_estimators 찾기 -> 3000

In [8]:
# model_rf = RandomForestClassifier(random_state = 2022)

In [9]:
# model_param_grid = {'n_estimators' : [100, 1000, 3000]} # Default(100)대비 큰값으로 Dictionary구성

In [10]:
# model_grid = GridSearchCV(model_rf, param_grid = model_param_grid, scoring="accuracy",n_jobs=-1,verbose=1) #훈련 (장치 사양에 따라 시간이 걸릴 수 있음)
# model_grid.fit(x_train, y_train)

In [11]:
# cv=KFold(n_splits=6, random_state=2022, shuffle=True) #교차검증

In [12]:
# gcv=GridSearchCV(model_rf, param_grid=model_param_grid, cv=cv, scoring='f1', n_jobs=-1)

In [13]:
# gcv.fit(x_train.values,y_train.values)
# print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력

# 최적의 n_estimators를 기반으로 model 생성

In [14]:
model = RandomForestClassifier(n_estimators=3000, random_state=2022, n_jobs=-1)

In [15]:
model.fit(x_train, y_train) #학습

RandomForestClassifier(n_estimators=3000, n_jobs=-1, random_state=2022)

In [16]:
pred = model.predict_proba(x_test)[:,1] #proba를 통해 확률값으로 변환

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

In [18]:
rf_pred = model.predict(x_test)
print(roc_auc_score(y_test, rf_pred))
print(np.round(accuracy_score(y_test, rf_pred), 4)) # 소수점 4자리 반올림

0.7759064538760644
0.7827


In [19]:
# pred = model.predict_proba(x_test)[:,1] #proba를 통해 확률값으로 변환
print(roc_auc_score(y_test, pred))
# print(np.round(accuracy_score(y_test, pred), 4)) # 소수점 4자리 반올림

0.8656831092680289


In [20]:
# 학습결과 평가
print("Train_Accuracy : ", model.score(x_train, y_train))
print("Test_Accuracy : ", model.score(x_test, y_test), '\n')

accuracy = mt.accuracy_score(y_test, rf_pred)
print("Accuracy :" ,mt.accuracy_score(y_test, rf_pred))

Train_Accuracy :  1.0
Test_Accuracy :  0.7826666666666666 

Accuracy : 0.7826666666666666


In [21]:
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

# kfold = StratifiedKFold(n_splits=5)
# cv_accuracy = []
# n_iter =0

# for train_index, test_index in kfold.split(train_x, train_y):  # feautres 데이터를 위에서 지정한 kfold 숫자로 분할
#     x_valid, x_test_fold = train_x.iloc[train_index], train_x.iloc[test_index]
#     y_valid, y_test_fold = train_y[train_index], train_y[test_index]
    
#     model_rf.fit(x_valid, y_valid)
#     pred = model_rf.predict(x_test_fold)
#     n_iter += 1
    
#     accuracy = np.round(accuracy_score(y_test_fold, pred), 4) # 소수점 4자리 반올림
#     train_size = x_valid.shape[0]
#     test_size = x_test_fold.shape[0]
    
#     print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
#           .format(n_iter, accuracy, train_size, test_size))
#     print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
#     cv_accuracy.append(accuracy)
    
# print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

In [22]:
# 학습한 모델로 예측수행
#test_pred = model.predict(test)
test_pred = model.predict_proba(test)[:,1] #proba를 통해 확률값으로 변환


In [23]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [24]:
submission["nerdiness"] = test_pred

In [25]:
submission

,index,nerdiness
0,0,0.693667
1,1,0.820667
2,2,0.839667
3,3,0.604000
4,4,0.843667
...,...,...
35447,35447,0.856333
35448,35448,0.705333
35449,35449,0.930000
35450,35450,0.607333


In [27]:
submission.to_csv("/content/drive/MyDrive/Dacon/RandomForest.csv", index = False)